# Naive RNN Forecast on Financial Data

### Description
This file is intended to demonstrate a simple and believably inaccurate version of RNN forecast on financial data. The data passed into the RNN model will only involve simple feature engineering.

### Imports

In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

### Read Data

In [85]:
# Data downloaded from investing.com and transformed by 
df = pd.read_csv('HSI Futures.csv', index_col='Date')

In [86]:
df

,Close,Open,High,Low
Date,,,,
2009-11-19,22537,22752,22853,22537
2009-11-20,22408,22543,22599,22357
2009-11-23,22772,22514,22805,22486
2009-11-24,22476,22785,22819,22263
2009-11-25,22606,22462,22680,22356
...,...,...,...,...
2020-08-17,25293,25052,25574,25014
2020-08-18,25341,25330,25440,25213
2020-08-19,25135,25349,25395,25041


## Method 1: Only Return Data
Use previous returns to predict the direction.

In [98]:
df1 = df.copy()

In [99]:
df1['return'] = (df1['Close'] / df1.shift(1)['Close']) - 1

In [100]:
df1['target'] = np.sign(df1['return']).apply(lambda x: 0 if x == -1.0 else x)

In [103]:
df1.dropna(inplace=True)
df1.drop(['Close', 'Open', 'High', 'Low'], axis=1, inplace=True)

### Define features and target

In [105]:
df1x = df1['return']
df1y = df1['target']

In [116]:
T = 20
D = 1
N = len(df1) - T
split = N // 3 * 2

X = []
Y = []

for t in range(N):
    X.append(df1x.iloc[t:t+T])
    Y.append(df1y.iloc[t+T])
    
X = np.array(X).reshape(N, T, D)
Y = np.array(Y)

### Model Building

In [115]:
model = Sequential([
    Input((T, D)),
    SimpleRNN(5),
    Dense(1, activation='relu')
])

model.compile(optimizer=Adam(lr=0.01), loss='mse', metrics=['accuracy'])

In [119]:
r1 = model.fit(X[:split], Y[:split], epochs=100, batch_size=10, validation_data=(X[split:], Y[split:]))

Train on 1752 samples, validate on 876 samples
Epoch 1/100
1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2510 - accuracy: 0.5171 - val_loss: 0.2490 - val_accuracy: 0.5331
Epoch 2/100
1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2527 - accuracy: 0.5023 - val_loss: 0.2504 - val_accuracy: 0.5331
Epoch 3/100
1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2520 - accuracy: 0.5234 - val_loss: 0.2685 - val_accuracy: 0.4669
Epoch 4/100
1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2514 - accuracy: 0.5114 - val_loss: 0.2490 - val_accuracy: 0.5331
Epoch 5/100
1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2513 - accuracy: 0.4943 - val_loss: 0.2490 - val_accuracy: 0.5331
Epoch 6/100
1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2506 - accuracy: 0.5183 - val_loss: 0.2546 - val_accuracy: 0.4669
Epoch 7/100
1752/1752 [==============================] - 3s 1ms/sample 

1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2518 - accuracy: 0.5091 - val_loss: 0.2500 - val_accuracy: 0.5331
Epoch 57/100
1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2503 - accuracy: 0.5126 - val_loss: 0.2499 - val_accuracy: 0.5342
Epoch 58/100
1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2525 - accuracy: 0.5023 - val_loss: 0.2489 - val_accuracy: 0.5331
Epoch 59/100
1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2504 - accuracy: 0.5177 - val_loss: 0.2498 - val_accuracy: 0.5331
Epoch 60/100
1752/1752 [==============================] - 3s 1ms/sample - loss: 0.2510 - accuracy: 0.4932 - val_loss: 0.2509 - val_accuracy: 0.5331
Epoch 61/100
1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2525 - accuracy: 0.5051 - val_loss: 0.2504 - val_accuracy: 0.5331
Epoch 62/100
1752/1752 [==============================] - 3s 2ms/sample - loss: 0.2516 - accuracy: 0.5011 - val_loss: 0.2499 

## Method 2: OHLC Data
OHLC stands for Open, High, Low and Close. In machine learning, regression prediction is a harder job compared to classification because of the difficulty to predict continuous value and to analyse the results, e.g. accuracy. Therefore, the target will be transformed into a daily return direction, namely positive (1) or negative (0).

In addition, in finance, the value of the index does not carry much prediction power per se. Thus the high, low and open data will be transformed into proportions to close.

One extra column on daily returns will be added.

In [89]:
df2 = df.copy()

In [90]:
df2['return'] = (df2['Close'] / df2.shift(1)['Close']) - 1

In [91]:
df2['target'] = np.sign(df2['return']).apply(lambda x: 0 if x == -1.0 else x)

In [92]:
for col in ['Open', 'High', 'Low']:
    df2[col] = df2[col] / df2['Close'] - 1

In [93]:
df2.dropna(inplace=True)
df2.drop('Close', axis=1, inplace=True)

In [94]:
df2

,Open,High,Low,return,target
Date,,,,,
2009-11-20,0.006025,0.008524,-0.002276,-0.005724,0.0
2009-11-23,-0.011330,0.001449,-0.012559,0.016244,1.0
2009-11-24,0.013748,0.015261,-0.009477,-0.012998,0.0
2009-11-25,-0.006370,0.003273,-0.011059,0.005784,1.0
2009-11-26,0.021425,0.021425,-0.003563,-0.019287,0.0
...,...,...,...,...,...
2020-08-17,-0.009528,0.011110,-0.011031,0.008734,1.0
2020-08-18,-0.000434,0.003907,-0.005051,0.001898,1.0
2020-08-19,0.008514,0.010344,-0.003740,-0.008129,0.0


### Define features and target

In [95]:
df2x = df2.drop('target', axis=1)
df2y = df2['target']

In [97]:
T = 20
D = 5
N = len(df2) - T

np.array(df2x.iloc[0:10])

array([[ 0.00602463,  0.00852374, -0.00227597, -0.00572392],
       [-0.0113297 ,  0.00144915, -0.01255928,  0.0162442 ],
       [ 0.013748  ,  0.01526072, -0.00947678, -0.01299842],
       [-0.00636999,  0.00327347, -0.01105901,  0.00578395],
       [ 0.02142535,  0.02142535, -0.00356337, -0.01928691],
       [ 0.0459564 ,  0.0459564 , -0.01104086, -0.04402346],
       [-0.00161053,  0.00924903, -0.00501565,  0.02538454],
       [-0.02564563,  0.        , -0.02564563,  0.02452604],
       [-0.00236512,  0.00205275, -0.00597974,  0.00646755],
       [-0.00492655,  0.00466025, -0.00772269,  0.00544424]])